In [1]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

c:\Users\samar\anaconda3\envs\agent_env\Lib\site-packages\langchain_core\_api\deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
load_dotenv()
model = ChatGroq(model="llama-3.3-70b-versatile", temperature=1)


In [3]:
class JokeState(TypedDict):
    joke: str
    topic:str
    explanation: str

In [5]:
def generate_joke(state:JokeState):
    joke = f"You are a comedian. Generate a joke about {state['topic']}."
    response = model.invoke(joke)
    return {'joke': response}

In [7]:
def explain_joke(state:JokeState):
    explanation_prompt = f"Explain the joke: {state['joke']}"
    response = model.invoke(explanation_prompt)
    return {'explanation': response}

In [10]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke',generate_joke)
graph.add_node('explain_joke', explain_joke)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'explain_joke')
graph.add_edge('explain_joke', END)

checkpoint_saver = InMemorySaver()

workflow = graph.compile(checkpoint_saver)


In [15]:
config1 = {'configurable':{'thread_id': '1'}}
intital_state = {'topic':'male friendship'}
workflow.invoke(intital_state,config=config1)

{'joke': AIMessage(content='"You know what they say about male friendships – we\'re like pizza, even when we\'re bad, we\'re still pretty good. But seriously, have you ever noticed that guys only show affection to each other when they\'re drunk, watching sports, or trying to lift something heavy? It\'s like, \'I love you, bro... now help me move this couch, I think I threw my back out.\'"', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 47, 'total_tokens': 130, 'completion_time': 0.324373563, 'completion_tokens_details': None, 'prompt_time': 0.002468204, 'prompt_tokens_details': None, 'queue_time': 0.048624109, 'total_time': 0.326841767}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c36e5-880f-7aa1-9863-e465efb44e30-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens':

In [16]:
workflow.get_state(config1)

StateSnapshot(values={'joke': AIMessage(content='"You know what they say about male friendships – we\'re like pizza, even when we\'re bad, we\'re still pretty good. But seriously, have you ever noticed that guys only show affection to each other when they\'re drunk, watching sports, or trying to lift something heavy? It\'s like, \'I love you, bro... now help me move this couch, I think I threw my back out.\'"', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 47, 'total_tokens': 130, 'completion_time': 0.324373563, 'completion_tokens_details': None, 'prompt_time': 0.002468204, 'prompt_tokens_details': None, 'queue_time': 0.048624109, 'total_time': 0.326841767}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c36e5-880f-7aa1-9863-e465efb44e30-0', tool_calls=[], invalid_tool_calls=[], usage_meta

In [14]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'joke': AIMessage(content='Why do programmers prefer dark mode? Because light attracts bugs. (get it?)', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 46, 'total_tokens': 63, 'completion_time': 0.073920875, 'completion_tokens_details': None, 'prompt_time': 0.002226682, 'prompt_tokens_details': None, 'queue_time': 0.049864488, 'total_time': 0.076147557}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c36d4-d728-7ef1-be1a-ea929270ecdb-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 46, 'output_tokens': 17, 'total_tokens': 63}), 'topic': 'programming', 'explanation': AIMessage(content='A joke about programmers and dark mode.\n\nThe joke is: "Why do programmers prefer dark mode? Because light attracts bugs."\n\nThe word "bugs" has a double meani